In [1]:
import numpy as np

from data_path import DATA_PATH_SROK8C

In [20]:
file = open(DATA_PATH_SROK8C+"20046.dat")

In [61]:
line = file.readline()
line = file.readline()
line = file.readline()
line = file.readline()
line = file.readline()
line = file.readline()
line = file.readline()


In [72]:
line[112:115]

'140'

In [ ]:
VALID_QUALITIES = [0, 1, 2]

def dictFromSrok8cDatFile(stationSynopticIndex: int):
    d = {
        "GMT year" : list(),
        "GMT month" : list(),
        "GMT day" : list(),
        "GMT time" : list(),
        "Horizontal visibility (km)" : list(),
        "Total cloudiness (points)" : list(),
        "Amount of low-level clouds (points)" : list(),
        "Shape of upper-level clouds" : list(),
        "Shape of middle-level clouds" : list(),
        "Shape of vertical development clouds" : list(),
        "Stratus and stratocumulus" : list(),
        "Nimbostratus and fracto-nimbus" : list(),
        "Cloud ceiling (m)" : list(),
        "Weather between observation hours" : list(),
        "Weather during observation hour" : list(),
        "Wind direction (degrees)" : list(),
        "Wind speed (m/s)" : list(),
        "Maximum wind speed (m/s)" : list(),
        "Precipitation total in the period between observation hours (mm)" : list(),
        "Soil surface temperature (°C)" : list(),
        "Minimum soil surface temperature (°C)" : list(),
        "Minimum soil surface temperature between observation hours (°C)" : list(),
        "Maximum soil surface temperature between observation hours (°C)" : list(),
        "Maximum thermometer temperature of soil surface (°C)" : list(),
        "Dry-bulb thermometer air temperature (°C)" : list(),
        "Wet-bulb thermometer air temperature (°C)" : list(),
        "Minimum air temperature between observation hours (°C)" : list(),
        "Maximum air temperature between observation hours (°C)" : list(),
        "Maximum thermometer air temperature (°C)" : list(),
        "Water vapour partial pressure (mbar)" : list(),
        "Relative air humidity (%)" : list(),
        "Water vapour saturation deficit (mbar)" : list(),
        "Dew point temperature (°C)" : list(),
        "Air pressure at station level (mbar)" : list(),
        "Air pressure at sea level (mbar)" : list(),
        "Characteristic of baric tendency" : list(),
        "Pressure tendency value (mbar)" : list()
    }
    
    with open(
        DATA_PATH_SROK8C
        + f"{stationSynopticIndex}.dat"
    ) as f:
        for line in f:
            # 0:4
            d["GMT year"] += [int(line[6:10])]
            d["GMT month"] += [int(line[11:13])]
            d["GMT day"] += [int(line[14:16])]
            d["GMT time"] += [int(line[17:19])]
            # 20:24 Year of source (local)
            # 25:27 Month of source (local)
            # 28:30 Day of source (local)
            # 31:33 Hour of source
            # 34:36 Number of hour in a day according to standard winter decree time (SWDT)
            # 37:39 Local time
            # 40:42 Number of time zone
            # 43:45 Meteorological day onset (SWDT)
            code = line[46:48]
            quality = int(line[49:50])
            idGT = line[51:52]
            d["Horizontal visibility (km)"] += [hrzVsbltInKm(code, quality, idGT)]
            code = line[53:55]
            quality = int(line[56:57])
            d["Total cloudiness (points)"] += [cldnssInPoints(code, quality)]
            code = line[58:60]
            quality = int(line[61:62])
            d["Amount of low-level clouds (points)"] += [cldnssInPoints(code, quality)]
            code = line[63:65]
            quality = int(line[66:67])
            d["Shape of upper-level clouds"] += [shpCld(code, quality)]
            code = line[68:70]
            quality = int(line[71:72])
            d["Shape of middle-level clouds"] += [shpCld(code, quality)]
            code = line[73:75]
            quality = int(line[76:77])
            d["Shape of vertical development clouds"] += [shpCld(code, quality)]
            code = line[78:80]
            quality = int(line[81:82])
            d["Stratus and stratocumulus"] += [shpCld(code, quality)]
            code = line[83:85]
            quality = int(line[86:87])
            d["Nimbostratus and fracto-nimbus"] += [shpCld(code, quality)]
            code = line[88:92]
            quality = int(line[93:94])
            method = line[95:96]
            d["Cloud ceiling (m)"] += [cldClng(code, quality, method)]
            # 97:99
            # 100:101
            code = line[102:104]
            quality = int(line[105:106])
            d["Weather between observation hours"] += [wthr(code, quality)]
            code = line[107:109]
            quality = int(line[110:111])
            d["Weather during observation hour"] += [wthr(code, quality)]
            code = line[112:105]
            quality = int(line[110:111])
            d["Wind direction (degrees)"] += [windDir(code, quality)]
            code = line[112:115]
            quality = int(line[116:117])
            idGT = line[118:119]
            d["Wind speed (m/s)"] += [windSpd(code, quality, idGT)]
            

def windSpd(code: str, quality: int, idGT: str):
    res = np.nan
    if quality in VALID_QUALITIES and idGT == "0":
        res = int(code)
    return res
            
def windDir(code: str, quality: int):
    NO_WIND = np.nan
    DIFF_DIRS = np.nan
    res = np.nan
    if quality in VALID_QUALITIES:
        if int(code) > 0 and int(code) <= 360:
            res = int(code)
        elif int(code) == 0:
            res = NO_WIND
        elif int(code) == 999:
            res = DIFF_DIRS
    return res    

def wthr(code: str, quality: int):
    res = np.nan
    if quality in VALID_QUALITIES:
        res = code
    return res                
            
def cldClng(code: str, quality: int, method: str):
    res = np.nan
    if quality in VALID_QUALITIES and code != "    ":
        res = int(code)

def shpCld(code: str, quality: int):
    res = np.nan
    if quality in VALID_QUALITIES:
        res = int(code)
    return res    
    
def cldnssInPoints(code: str, quality: int):
    CLDNSS_11 = np.nan #11
    CLDNSS_12 = np.nan #12
    CLDNSS_13 = np.nan #13
    res = np.nan
    if quality in VALID_QUALITIES and code != "99":
        if code == "11":
            res = CLDNSS_11
        elif code == "12":
            res = CLDNSS_12
        elif code == "13":
            res = CLDNSS_13
        else:
            res = int(code)
    return res        

def hrzVsbltInKm(code: str, quality: int, idGT: str):
    LT_01KM = np.nan #0.1 * (1.0 - 0.5)
    LT_005KM = np.nan #0.05 * (1.0 - 0.5)
    GT_70KM = np.nan #70.0 * (1.0 + 0.5)
    GT_50KM = np.nan #50.0 * (1.0 + 0.5)
    res = np.nan
    if quality in VALID_QUALITIES and idGT == " ":
        if code == "00":
            res = LT_01KM
        elif code in genCodes(1, 50+1):
            res = float(int(code)) / 10.0
        elif code in genCodes(56, 80+1):
            res = float(code) - 50.0
        elif code in genCodes(81, 88+1):
            res = (float(code) - 76.0) * 5.0
        elif code == "89":
            res = GT_70KM
        elif code == "90":
            res = LT_005KM
        elif code == "91":
            res = 0.05
        elif code == "92":
            res = 0.2
        elif code == "93":
            res = 0.3
        elif code == "94":
            res = 1.0
        elif code == "95":
            res = 2.0
        elif code == "96":
            res = 4.0
        elif code == "97":
            res = 10.0
        elif code == "98":
            res = 20.0
        elif code == "99":
            res = GT_50KM    
    return res

def genCodes(start, end):
    res = list()
    if end > 99:
        return np.nan
    for i in range(start, end):
        if i < 10:
            res += [f"0{i}"]
        else:
            res += [f"{i}"]
    return res